In [2]:
import pymongo
import torch
import torchvision

from superduperdb import superduper

In [3]:
class LeNet5(torch.nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=0),
            torch.nn.BatchNorm2d(6),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=0),
            torch.nn.BatchNorm2d(16),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = torch.nn.Linear(400, 120)
        self.relu = torch.nn.ReLU()
        self.fc1 = torch.nn.Linear(120, 84)
        self.relu1 = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(84, num_classes)

    def preprocess(self, x):
        return torchvision.transforms.Compose([
            torchvision.transforms.Resize((32, 32)),
            torchvision.transforms.ToTensor(),
            torchvision.transforms.Normalize(mean=(0.1307,), std=(0.3081,))]
        )(x)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        out = self.relu(out)
        out = self.fc1(out)
        out = self.relu1(out)
        out = self.fc2(out)
        return out

    def postprocess(self, x):
        return int(x.topk(1)[1].item())
    
model = LeNet5(10)

In [4]:
db = pymongo.MongoClient().my_db

In [5]:
db = superduper(db)

INFO:faiss.loader:Loading faiss.
INFO:faiss.loader:Successfully loaded faiss.


In [6]:
from superduperdb.types.pillow.image import pil_image as i
from superduperdb.core.documents import Document as D
from superduperdb.queries.mongodb.queries import Collection

import random

mnist_data = list(torchvision.datasets.MNIST(root='./data', download=True))
data = [D({'img': i(x[0]), 'class': x[1]}) for x in mnist_data]
random.shuffle(data)

db.execute(Collection(name='mnist').insert_many(data[:-1000], encoders=[i]))

Stripping sub-components to references


INFO:root:Created type/pil_image/0
INFO:root:found 0 uris
INFO:root:found 0 uris


(<pymongo.results.InsertManyResult at 0x1668b4c10>,
 TaskWorkflow(database=<superduperdb.datalayer.base.database.BaseDatabase object at 0x16961b640>, G=<networkx.classes.digraph.DiGraph object at 0x1668b4580>))

In [19]:
r = db.execute(
    Collection(name='mnist').find_one()
)

r['img'].x

In [20]:
r.encode()

{'_id': ObjectId('649ec8aafea5ef9291213e2c'),
 'img': {'_content': {'bytes': b"\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x00\x1c\x00\x00\x00\x1c\x08\x00\x00\x00\x00Wf\x80H\x00\x00\x01'IDATx\x9cc`\xa05\xe8\xf8\xfb7\x10\x87\x14\xab\xd9\xe3/\xb5\\\xd8\xe5\xd8\xda\xff\xecs\xc0ef\xd9\x9f\x8f\x06\xb8\xe4\xf4\x1f\xfeY\x8cK\x8ee\xfd\x9f\xeb\x82X\x84\x19\x18\x18\x18\x188}\x19&\xbf\x87\xb8+\x80\x81\x81a\xff\x1bd%\xa5\x7f\xd6\xf3@X\xd6\x7f\xfe\xfc\xf9\xf3p#\x1f\x92\xe4\xa7?\t\x10\x86\xe9\xa7O\xfd\xfd+\xff\xfcy\xe1\x89\x90\xfc\x0b\x93\xac\xfca\xc7\xc0\xc0\xcc=\xf5\xcfwW\xb8\xe4\xff\xbfP\xc9Oo!\xf4\xb4?\xfb\x18\x18\x18\x98\x18\x18\x18\x18\x18\xfe\xfd\x87\xaa\xfar\x11B\x97\xbc>\n\x97\x84\x83\xc9\xc6\x0c\x0c\x0c\x0c\x0c\xbc\xcbX\xb7\xc0%\xd73\xc8\t@d\x99E\x19\x18\x18\xf8\x16*\xf9\x9f\x84k\xd0y\xf4g\xab\x1e\x03\x03\x03C\xe5\xdf\x0eN[\xdb\xb5\x8bPb@o\xdf\x9fw\x07gxz\xce\xfb\xfb\xed\xe4\xbb-\x13yP-cw_\xfe\xe2\xcf\x9f?\x7f\xfe\xfe\xf9\x93\xc5\xc0\xc0\xc0\xe0\x9b\x86*/\xed\x93\xa7\xa2\xa2y\xf9\xcfY/\x06

In [11]:
model = superduper(model)

In [18]:
from superduperdb.models.torch import TorchTrainerConfiguration

training_configuration = TorchTrainerConfiguration(
    identifier='train_config',
    objective=torch.nn.functional.cross_entropy,
    max_iterations=1000,
    loader_kwargs=dict(
        batch_size=50,
        num_workers=0,
    )
)

In [ ]:
jobs = db.add(
    Watcher(
        model=model,
        key='img',
        depends_on=model.fit(
            X='img',
            y='class',
            database=db,
            select=Collection('mnist').find(),
        )
    )
)

In [ ]:
jobs[0].watch()

In [ ]:
from matplotlib import pyplot as plt

model = db.load('model', model.identifier)

plt.plot(model.metric_values['acc'])
plt.show()

In [ ]:
jobs[1].watch()

In [ ]:
for r in data[-1000:]:
    r['update'] = True

db.execute(Collection('docs').insert_many(data[-1000:]))

In [ ]:
db.execute(Collection('docs').find_one({'update': True}))